# Day 8: Resonant Collinearity

## Part One

You find yourselves on the roof of a top-secret Easter Bunny installation.

While The Historians do their thing, you take a look at the familiar huge antenna. Much to your surprise, it seems to have been reconfigured to emit a signal that makes people 0.1% more likely to buy Easter Bunny brand Imitation Mediocre Chocolate as a Christmas gift! Unthinkable!

Scanning across the city, you find that there are actually many such antennas. Each antenna is tuned to a specific frequency indicated by a single lowercase letter, uppercase letter, or digit. You create a map (your puzzle input) of these antennas. For example:

```
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............
```

The signal only applies its nefarious effect at specific antinodes based on the resonant frequencies of the antennas. In particular, an antinode occurs at any point that is perfectly in line with two antennas of the same frequency - but only when one of the antennas is twice as far away as the other. This means that for any pair of antennas with the same frequency, there are two antinodes, one on either side of them.

So, for these two antennas with frequency a, they create the two antinodes marked with #:

```
..........
...#......
..........
....a.....
..........
.....a....
..........
......#...
..........
..........
```

Adding a third antenna with the same frequency creates several more antinodes. It would ideally add four antinodes, but two are off the right side of the map, so instead it adds only two:

```
..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......#...
..........
..........
```

Antennas with different frequencies don't create antinodes; A and a count as different frequencies. However, antinodes can occur at locations that contain antennas. In this diagram, the lone antenna with frequency capital A creates no antinodes but has a lowercase-a-frequency antinode at its location:

```
..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......A...
..........
..........
```

The first example has antennas with two different frequencies, so the antinodes they create look like this, plus an antinode overlapping the topmost A-frequency antenna:

```
......#....#
...#....0...
....#0....#.
..#....0....
....0....#..
.#....A.....
...#........
#......#....
........A...
.........A..
..........#.
..........#.
```

Because the topmost A-frequency antenna overlaps with a 0-frequency antinode, there are `14` total unique locations that contain an antinode within the bounds of the map.

Calculate the impact of the signal. How many unique locations within the bounds of the map contain an antinode?

In [1]:
const input = true ?
	await Deno.readTextFile("./work/2024/08/day8input.txt") :
	`
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............`.trim()

## Part One Plan

Create a map of the positions of each antenna keyed by the character representing it.
Each map entry will contain a list of `<row,col>` positions for each antenna.

Iterating through the map, calculate the distance between each antenna position (vector), 
apply the vector to the subject of the iteration, 
mark the antinodes on a copy of the original input with a `#`,
and then finally count the number of `#` in the string.
Alternatively, increment a counter when an antinode fits within the confines of the square
grid, but it might be useful to visualize the antinodes that are identified by the program.


In [2]:
const rows:string[] = input.split("\n")
const dimensions:[int,int] = [rows.length, rows[0].length]

type Position = [int, int]
type PositionKey = `${string},${string}`
function positionKeyToPosition(key:positionKey): Position {
	return key.split(',').map(a => parseInt(a, 10))
}
const antennaPositions = new Map<string,PositionKey[]>()
rows.forEach((row:string, rowi:int) => {
	row.split('').forEach((sym:string, coli:int) => {
		if (sym.match(/[\w\d]/)) {
			if (!antennaPositions.has(sym)) {
				antennaPositions.set(sym, [`${rowi},${coli}`])
			} else {
				antennaPositions.get(sym)?.push(`${rowi},${coli}`)
			}
		}
	})
})
let rowsCopy:string[] = [...rows]
antennaPositions.forEach((pos:PositionKey[], antenna:string) => {
	pos.forEach((subject:PositionKey, posi) => {
		pos.forEach((outerPosition:PositionKey, oposi) => {
			let [subr, subc] = positionKeyToPosition(subject)
			let [outr, outc] = positionKeyToPosition(outerPosition)
			let vector:Position = [subr - outr, subc - outc]
			let antinode:Position = [subr + vector[0], subc + vector[1]]
			if (
				!pos.includes(antinode.join(','))
				&& antinode[0] >= 0
				&& antinode[1] >= 0
				&& antinode[0] < dimensions[0]
				&& antinode[1] < dimensions[1]
			) {
				let row = rowsCopy[antinode[0]].split('')
				row[antinode[1]] = '#'
				rowsCopy[antinode[0]] = row.join('')
			}
		})
	})
})
let antinodeCount = rowsCopy.reduce((sum:int, row:string) : int => {
	return sum + (row.match(/#/g)?.length || 0)
}, 0)
console.log(`Antinode count: ${antinodeCount}`)

Antinode count: 301


## Part Two

Watching over your shoulder as you work, one of The Historians asks if you took the effects of resonant harmonics into your calculations.

Whoops!

After updating your model, it turns out that an antinode occurs at any grid position exactly in line with at least two antennas of the same frequency, regardless of distance. This means that some of the new antinodes will occur at the position of each antenna (unless that antenna is the only one of its frequency).

So, these three T-frequency antennas now create many antinodes:

```
T....#....
...T......
.T....#...
.........#
..#.......
..........
...#......
..........
....#.....
..........
```

In fact, the three T-frequency antennas are all exactly in line with two antennas, so they are all also antinodes! This brings the total number of antinodes in the above example to 9.

The original example now has 34 antinodes, including the antinodes that appear on every antenna:

```
##....#....#
.#.#....0...
..#.#0....#.
..##...0....
....0....#..
.#...#A....#
...#..#.....
#....#.#....
..#.....A...
....#....A..
.#........#.
...#......##
```

Calculate the impact of the signal using this updated model. How many unique locations within the bounds of the map contain an antinode?

## Part Two Plan

The approach to this part is very similar, the only difference is instead of looking for an antinode one "vector" away from the subject, keep marking the antinodes until we reach the edge of the map!

In [3]:
rowsCopy = [...rows]

antennaPositions.forEach((pos: PositionKey[], antenna: string) => {
	pos.forEach((subject: PositionKey, posi) => {
		pos.forEach((outerPosition: PositionKey, oposi) => {
			let [subr, subc] = positionKeyToPosition(subject)
			let [outr, outc] = positionKeyToPosition(outerPosition)
			let vector: Position = [subr - outr, subc - outc]
			let antinode: Position = [subr + vector[0], subc + vector[1]]
			while (
				vector.join(',') != '0,0' // subject is the same as outer position
				&& antinode[0] >= 0
				&& antinode[1] >= 0
				&& antinode[0] < dimensions[0]
				&& antinode[1] < dimensions[1]
			) {
				if (!pos.includes(antinode.join(','))) {
					let row = rowsCopy[antinode[0]].split('')
					row[antinode[1]] = '#'
					rowsCopy[antinode[0]] = row.join('')
				}
				antinode[0] += vector[0]
				antinode[1] += vector[1]
			}
		})
	})
})

let antinodeCount = rowsCopy.reduce((sum: int, row: string): int => {
	return sum + (row.match(/[^\.]/g)?.length || 0)
}, 0)
console.log(`Antinode count with updated model: ${antinodeCount}`)

Antinode count with updated model: 1019
